In [ ]:
import numpy as np
import time as time
import os
import glob
import copy

# Set up matplotlib and use a nicer set of plot parameters
# %config InlineBackend.rc = {}
import matplotlib as mpl
# mpl.rc('mathtext',fontset='stixsans')
# mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook
import scipy.optimize
import tqdm

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

import constants
import lyafxcorr_kg

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
#PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
#SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

PiBound = (min(PiEdges), max(PiEdges) )

In [ ]:
OBS_DIR = os.path.join(constants.XCORR_DIR_BASE, 'obs')

XCorr_3d=np.load(os.path.join(OBS_DIR, f"xcorr_3dhst_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_3d = np.transpose(XCorr_3d)
    
XCorr_zD = np.load(os.path.join(OBS_DIR, f"xcorr_zDeep_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_zD = np.transpose(XCorr_zD)
    
XCorr_mosdef = np.load(os.path.join(OBS_DIR, f"xcorr_mosdef_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_mosdef = np.transpose(XCorr_mosdef)
    
XCorr_vuds = np.load(os.path.join(OBS_DIR, f"xcorr_vuds_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_vuds = np.transpose(XCorr_vuds)

XCorr_clamato = np.load(os.path.join(OBS_DIR, f"xcorr_clamato_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_clamato = np.transpose(XCorr_clamato)

XCorr_all = np.load(os.path.join(OBS_DIR, f"xcorr_all_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_all = np.transpose(XCorr_all)

print(XCorr_all.shape)

xcorr_list = [XCorr_3d, XCorr_zD, XCorr_mosdef, XCorr_vuds, XCorr_clamato]
titles = ['3DHST', 'zDeep', 'MOSDEF', 'VUDS', 'CLAMATO']

In [ ]:
COVAR_DIR = os.path.join(constants.XCORR_DIR_BASE, 'bootstrap')

Covar_zdeep = np.load(os.path.join(COVAR_DIR, f'covar_zDeep_n7200_{constants.DATA_VERSION}.npy'))
Covar_vuds  = np.load(os.path.join(COVAR_DIR, f'covar_VUDS_n7200_{constants.DATA_VERSION}.npy'))
Covar_clamato = np.load(os.path.join(COVAR_DIR, f'covar_CLAMATO_n7200_{constants.DATA_VERSION}.npy'))
Covar_mosdef = np.load(os.path.join(COVAR_DIR, f'covar_MOSDEF_n7200_{constants.DATA_VERSION}.npy'))
Covar_3dhst = np.load(os.path.join(COVAR_DIR, f'covar_3DHST_n7200_{constants.DATA_VERSION}.npy'))

covar_list = [Covar_3dhst, Covar_zdeep, Covar_mosdef, Covar_vuds, Covar_clamato]

In [ ]:
parallel_bin_centers = []
for i in range(len(PiEdges) - 1):
    lb, ub = PiEdges[i], PiEdges[i+1]
    parallel_bin_centers.append(np.mean([lb, ub]))
parallel_bin_centers = np.array(parallel_bin_centers)

In [ ]:
mask = np.ones_like(parallel_bin_centers)
mask[:5] = 0
mask[-5:] = 0
mask = mask.astype(bool)
print(np.max(parallel_bin_centers[mask]) - np.min(parallel_bin_centers[mask]))

In [ ]:
# Reduce covar in adding.
def reduce_covar(covar):
    parallel_size = XCorr_all.shape[1]
    if covar.ndim != 4:
        if covar.shape[0] == XCorr_all.size:
            covar = covar.reshape(*XCorr_all.shape, *XCorr_all.shape)
        else:
            raise RuntimeError
    # Verified through linearity of covariance when summing that this is the correct procedure.
    return np.sum(covar, axis=(0, 2))
    
reduced_covar_list = []
for covar in covar_list:
    reduced_covar_list.append(reduce_covar(np.copy(covar)))
    assert np.allclose(reduced_covar_list[-1], reduced_covar_list[-1].T)

In [ ]:
def peak(x, *p):
    A, mu, C = p
    sigma = 30
    dist = np.minimum(np.abs(x - (mu + sigma)), np.abs(x - (mu - sigma)))
    dist[x > mu + sigma] = 0
    dist[x < mu - sigma] = 0
    return A * dist**3 + C
    #return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + C
    
def peak(x, *p):
    A, mu, C = p
    sigma = 30
    dist = np.minimum(np.abs(x - (mu + sigma)), np.abs(x - (mu - sigma)))
    dist[x > mu + sigma] = 0
    dist[x < mu - sigma] = 0
    return A * dist**3 + C
    #return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + C

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(11, 6))

for xcorr, covar, t, ax in zip(xcorr_list, reduced_covar_list, titles, axes.flatten()):
    reduced_xcorr = np.sum(xcorr, axis=0)
    # covar = np.diag(np.diag(covar))
    ax.errorbar(parallel_bin_centers, reduced_xcorr, yerr=np.diag(covar)**0.5, color='black')
    background = np.mean([reduced_xcorr[0], reduced_xcorr[-1]])
    fit_coeff, _ = scipy.optimize.curve_fit(peak, parallel_bin_centers, reduced_xcorr, p0=[(np.min(reduced_xcorr) - background) / 30**3, 0, background], sigma=covar)
    print(fit_coeff)
    ax.plot(parallel_bin_centers, peak(parallel_bin_centers, *fit_coeff), color='red')
    ax.set_title(f'{t} delta={-fit_coeff[1]:.2}')

In [ ]:
plt.imshow(reduced_covar_list[1])

# Mock validation

In [ ]:
def fit_deltaz_validation(xcorr, covar, mask, covar_diag):
    reduced_xcorr = np.sum(xcorr, axis=0)
    x = parallel_bin_centers[mask]
    reduced_xcorr = reduced_xcorr[mask]
    covar = covar[mask, :][:, mask]
    if covar_diag:
        covar = np.diag(covar)
    background = np.mean([reduced_xcorr[0], reduced_xcorr[-1]])
    fit_coeff, _ = scipy.optimize.curve_fit(peak, x, reduced_xcorr, 
                                            p0=[(np.min(reduced_xcorr) - background) / 30**3, 0, background], 
                                            sigma=covar,
                                            maxfev=10000)
    return fit_coeff[1]

def fit_plot_deltaz_mock(survey_name, mask=None, covar_diag=True):
    N_VOL = 63
    N_ABS = 3
    N_GAL = 10
    if mask is None:
        mask = np.ones_like(parallel_bin_centers).astype(bool)
    deltaz = []
    # covar = reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_{survey_name}_v0.npy')))
    for igal in tqdm.tqdm(range(N_GAL)):
        for iabs in range(N_ABS):
            for ivol in (range(1, N_VOL + 1)):
                abssuffix = '{0:03d}'.format(ivol+iabs*N_VOL)
                galsuffix = '{:03d}'.format(ivol+igal*N_VOL)
                xcorr_data = np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr', f'xcorrmock_{survey_name}_g{galsuffix}_a{abssuffix}_v0.npy'))
                
#                 lyafil = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'skewers', 'pixel_radecz_mock_'+abssuffix+'.bin')
#                 lyapix = lyafxcorr_kg.lyapix(lyafil,cosmo=cosmo)
#                 lyapix_skewerrec = lyapix.gen_SkewerRec()
#                 npix = lyapix.npix

#                 galfil = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'gal', 'cat_galmock_nonuniq_'+galsuffix+'.dat')
#                 gal = ascii.read(galfil, format='ipac')
#                 assert gal['ra'].unit == u.degree
#                 assert gal['dec'].unit == u.degree
#                 source_mask = np.array([s.lower() == survey_name.lower() for s in gal['source']])
#                 gal = gal[source_mask]
#                 ngal = len(gal)
#                 Coord = SkyCoord(ra=gal['ra'], dec=gal['dec'], distance=cosmo.comoving_distance(gal['zspec']))
                
#                 def do_resampling(n_sub_partial, seed=None):
#                     XCorrSamples_Partial = np.zeros([len(SigEdges)-1, len(PiEdges)-1, n_sub_partial])
#                     rng = np.random.default_rng(seed)
#                     for ii in range(n_sub_partial):
#                         # Make a copy of the pixels and resample
#                         LyaPixTmp = copy.deepcopy(lyapix)
#                         LyaPixTmp.rng = rng
#                         LyaPixTmp = LyaPixTmp.resample_skewer(SkewerRec=lyapix_skewerrec)
#                         # Resample galaxy positions
#                         GalCoordTmp = Coord[rng.choice(ngal,ngal,replace=True)]
#                         XCorrTmp, _ = lyafxcorr_kg.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
#                                                           cosmo=cosmo,verbose=0)
#                         XCorrSamples_Partial[:, :, ii] = XCorrTmp
#                     return XCorrSamples_Partial
                
#                 XCorrSamples = do_resampling(100)
#                 boot_covar = np.cov(XCorrSamples)
                boot_covar = np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_{survey_name}_v0.npy'))
                boot_covar = reduce_covar(boot_covar)
                
                deltaz.append(fit_deltaz_validation(xcorr_data, boot_covar, mask, covar_diag))
    plt.hist(deltaz, bins=20)
    print(np.mean(deltaz), np.median(deltaz))
    plt.title(survey_name)
    plt.show()

In [ ]:
def average_xcorrs(survey_name):
    xcorr_path_list = glob.glob(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr', f"xcorrmock_{survey_name}_*_{constants.DATA_VERSION}.npy"))
    xcorr_plot_sum = np.zeros_like(XCorr_all)
    for i, p in enumerate(xcorr_path_list):
        xcorr_plot_sum += np.load(p)
    return xcorr_plot_sum / len(xcorr_path_list)

Avg_XCorr_mosdef = average_xcorrs('mosdef')
Avg_XCorr_vuds = average_xcorrs('vuds')
Avg_XCorr_zD = average_xcorrs('zDeep')
Avg_XCorr_3d = average_xcorrs('3dhst')
Avg_XCorr_clamato = average_xcorrs('clamato')

In [ ]:
# truth 0.8
fit_plot_deltaz_mock('mosdef', mask=None, covar_diag=True)
fit_deltaz_validation(Avg_XCorr_mosdef, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_mosdef_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      True)

In [ ]:
# truth 1.8
fit_plot_deltaz_mock('zDeep', mask=None)
fit_deltaz_validation(Avg_XCorr_zD, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_zDeep_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      True)

In [ ]:
# truth 1
fit_plot_deltaz_mock('vuds', mask=None)
fit_deltaz_validation(Avg_XCorr_vuds, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_vuds_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      True)

In [ ]:
# truth 1
fit_plot_deltaz_mock('clamato', mask=None, covar_diag=True)
fit_deltaz_validation(Avg_XCorr_clamato, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_clamato_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      True)

In [ ]:
# truth 1
fit_plot_deltaz_mock('3dhst', mask=None)
fit_deltaz_validation(Avg_XCorr_3d, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_3dhst_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      True)